# ARC - Data Source Credential Setup

This notebook helps you configure credentials for all ARC data sources.
Credentials are saved to `~/.arc/config.json` with restricted file permissions.

## Data Source Summary

| Source | Credentials | How to Get |
|--------|-------------|------------|
| **AWS Earth Search** | None needed | Just works |
| **Planetary Computer** | None needed | `pip install planetary-computer` |
| **CDSE** (default) | S3 keys or login | Free account at dataspace.copernicus.eu |
| **GEE** | Google account | `earthengine authenticate` |

## 1. Install ARC

In [ ]:
!pip install -q https://github.com/profLewis/ARC/archive/refs/heads/main.zip

## 2. Check current credential status

In [ ]:
from arc.credentials import print_config_status, load_config, save_config, CONFIG_FILE
print_config_status()

---
## 3. AWS Earth Search (no setup needed)

[Element 84 Earth Search](https://earth-search.aws.element84.com/v1) provides Sentinel-2 L2A as Cloud Optimized GeoTIFFs. **No account or credentials required.**

```python
arc.arc_field(..., data_source='aws')
```

In [ ]:
# Test AWS access (no credentials needed)
from pystac_client import Client
catalog = Client.open('https://earth-search.aws.element84.com/v1')
print(f'AWS Earth Search: OK ({catalog.title})')

---
## 4. Microsoft Planetary Computer (pip install only)

[Planetary Computer](https://planetarycomputer.microsoft.com/) serves Sentinel-2 as COGs. Authentication is handled automatically by the `planetary-computer` package.

```bash
pip install planetary-computer
```

```python
arc.arc_field(..., data_source='planetary')
```

In [ ]:
!pip install -q planetary-computer

try:
    import planetary_computer
    print(f'planetary-computer installed: v{planetary_computer.__version__}')
except ImportError:
    print('Failed to install planetary-computer')

---
## 5. CDSE (Copernicus Data Space Ecosystem)

CDSE is the **default** data source. Two authentication methods are supported.

### Getting CDSE credentials

**Method 1: S3 Access Keys (recommended)**
1. Register for a free account at [dataspace.copernicus.eu](https://dataspace.copernicus.eu)
2. Go to the [S3 credentials dashboard](https://eodata.dataspace.copernicus.eu)
3. Click **"Generate S3 credentials"** to create an access key and secret key
4. Enter them below

**Method 2: Username / Password**
1. Register at [dataspace.copernicus.eu](https://dataspace.copernicus.eu)
2. Use your login email and password

In [ ]:
import getpass

config = load_config()

print('CDSE Credential Setup')
print('=' * 40)
print('1: S3 Access Keys (recommended)')
print('2: Username / Password')
print('3: Skip')
method = input('Choose method [1/2/3]: ').strip()

if method == '1':
    config['cdse']['s3_access_key'] = getpass.getpass('CDSE S3 Access Key: ')
    config['cdse']['s3_secret_key'] = getpass.getpass('CDSE S3 Secret Key: ')
    config['cdse']['username'] = ''
    config['cdse']['password'] = ''
    save_config(config)
    print(f'S3 credentials saved to {CONFIG_FILE}')
elif method == '2':
    config['cdse']['s3_access_key'] = ''
    config['cdse']['s3_secret_key'] = ''
    config['cdse']['username'] = input('CDSE Username (email): ')
    config['cdse']['password'] = getpass.getpass('CDSE Password: ')
    save_config(config)
    print(f'Login credentials saved to {CONFIG_FILE}')
else:
    print('Skipped CDSE setup.')

In [ ]:
# Test CDSE credentials
from arc.credentials import get_cdse_credentials
creds = get_cdse_credentials()

if creds['s3_access_key'] and creds['s3_secret_key']:
    from osgeo import gdal
    from pystac_client import Client

    gdal.SetConfigOption('AWS_S3_ENDPOINT', 'eodata.dataspace.copernicus.eu')
    gdal.SetConfigOption('AWS_ACCESS_KEY_ID', creds['s3_access_key'])
    gdal.SetConfigOption('AWS_SECRET_ACCESS_KEY', creds['s3_secret_key'])
    gdal.SetConfigOption('AWS_VIRTUAL_HOSTING', 'FALSE')
    gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN', 'EMPTY_DIR')

    catalog = Client.open('https://stac.dataspace.copernicus.eu/v1')
    search = catalog.search(collections=['sentinel-2-l2a'], max_items=1)
    items = list(search.items())
    if items:
        item = items[0]
        href = item.assets['B02_10m'].href
        s3_path = '/' + href[5:] if href.startswith('s3://') else href
        ds = gdal.Open(f'/vsis3{s3_path}')
        if ds is not None:
            print(f'CDSE S3 access: OK (read {item.id} B02)')
            ds = None
        else:
            print('CDSE S3 access: FAILED - check your keys')
elif creds['username'] and creds['password']:
    import requests
    resp = requests.post(
        'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
        data={'grant_type': 'password', 'client_id': 'cdse-public',
              'username': creds['username'], 'password': creds['password']}
    )
    if resp.status_code == 200 and 'access_token' in resp.json():
        print('CDSE OAuth2: OK')
    else:
        print(f'CDSE OAuth2: FAILED (HTTP {resp.status_code})')
else:
    print('CDSE: not configured (run the cell above to set credentials)')

---
## 6. Google Earth Engine (GEE)

GEE authentication is managed by the `earthengine-api` package. Credentials are stored separately by Google.

### Getting GEE access
1. Sign up at [earthengine.google.com](https://earthengine.google.com/)
2. Run the cell below to authenticate

```python
arc.arc_field(..., data_source='gee')
```

In [ ]:
import ee

try:
    ee.Initialize()
    print('GEE: already authenticated')
except Exception:
    print('GEE: not authenticated. Running ee.Authenticate()...')
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
    print('GEE: authenticated successfully')

---
## 7. Set data source preference order

When using `data_source='auto'`, ARC picks the first available source from your preference list.
You can also run a speed probe to find the fastest source for your location.

In [ ]:
from arc.credentials import load_config, save_config, get_available_sources

config = load_config()
available = get_available_sources()

print(f"Available sources: {available}")
print(f"Current preference: {config.get('data_source_preference', [])}")
print()
print("Enter new preference order as comma-separated list, or press Enter to keep current:")
print(f"  Options: {', '.join(available)}")
order = input("Preference order: ").strip()
if order:
    new_pref = [s.strip() for s in order.split(',') if s.strip()]
    config['data_source_preference'] = new_pref
    save_config(config)
    print(f"Saved preference: {new_pref}")
else:
    print("Keeping current preference order.")

---
## 8. Final status check

In [ ]:
print_config_status()